In [1]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

print(torch.__version__)

1.12.1


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
env = gym.make("CartPole-v0")  ### 或者 env = gym.make("CartPole-v0").unwrapped 开启无锁定环境训练

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.001

In [ ]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns


def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters(), lr)
    optimizerC = optim.Adam(critic.parameters(), lr)
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            # env.render()
            state = torch.tensor(state, dtype=torch.float32).to(device)
            dist, value = actor(state), critic(state)   #演员的动作，评论家给予当前状态的价值

            action = dist.sample([1])
            next_state, reward, done, _ = env.step(action.cpu().squeeze(0).numpy()) 

            log_prob = dist.log_prob(action)        #返回该动作的可能性
            # entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)                    
            rewards.append(torch.tensor([reward], dtype=torch.float32).to(device))
            masks.append(torch.tensor([1-done], dtype=torch.float32).to(device))

            state = next_state

            if done:
                if iter % 10 == 0:
                    print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
        next_value = critic(next_state)             # 评论家对结局打分
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.concat(log_probs)
        returns = torch.concat(returns).detach()
        values = torch.concat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor.state_dict(), 'model/actor.pdparams')
    torch.save(critic.state_dict(), 'model/critic.pdparams')
    env.close()

In [96]:
print(1-False)

1
